In [1]:
import sys
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2

base_path=Path("d:/")
base_path = base_path / "work" / "projects"
for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import airsim

D:\work\projects\ezai_util
D:\work\projects\ai_coop_py


In [2]:
uenv_path = Path("d:/") / 'work' / 'unity-envs'
uenv_file = uenv_path / "airsim_win" / 'AirSimAssets'
uenv_file = str(uenv_file.resolve())

In [3]:
env = airsim.AirSimEnv(filename=uenv_file, observation_mode = 0)
env.info()

2020-10-12 23:41:30 INFO [environment.py:107] Connected to Unity environment with package version 1.1.0-preview and communication version 1.0.0
2020-10-12 23:41:30 INFO [environment.py:265] Connected new brain:
VisualNavigator?team=0


Env Info
-----------
Action Space: Box(2,)
Action sample: [ 0.39293838 -0.42772132]
Action Space Shape: (2,)
Action Space Low: [-1. -1.]
Action Space High: [1. 1.]
Observation Mode: 0
Gym Observation Space: Tuple(Box(10,))
Gym Observation Space Shape: None
Self Observation Space: Tuple(Box(10,))
Self Observation Space Shape: None
Reward Range: (-inf, inf)
Metadata: {'render.modes': ['rgb_array']}
Initial State: [array([ 2.3926267e+03,  3.7319370e+01, -3.7768066e+02, -2.9097432e-03,
        9.2058080e-01,  3.1531379e-03, -3.9052874e-01,  2.5062500e+03,
        4.1240845e+01, -3.6359070e+02], dtype=float32)]
First Step State: ([array([ 2.3926272e+03,  3.7317383e+01, -3.7768079e+02, -3.7698867e-03,
        9.2058468e-01,  5.0607547e-03, -3.9049214e-01,  2.5062500e+03,
        4.1240845e+01, -3.6359070e+02], dtype=float32)], -1e-04, False, {'step': <mlagents_envs.base_env.DecisionSteps object at 0x00000289CCCA52E0>})


c:\Miniconda3\envs\ezai\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
from ezai_util import DictObj

In [5]:
model_conf = DictObj()
model_conf.n_x = env.n_x
model_conf.n_y = env.n_y
model_conf.n_batch_size = 64
model_conf.n_epochs = 10

policy_conf = DictObj()
policy_conf.discount_rate=0.9
policy_conf.learning_rate=0.8
policy_conf.explore_rate_max=0.9
policy_conf.explore_rate_min=0.01
policy_conf.explore_rate_decay=0.005
policy_conf.memory_capacity = model_conf.n_batch_size * 10

rl_conf = DictObj({
  "env_name":'airsim',
  "seed": 123,
  "discount": 0.99,
  "tau": 5e-3,
  "time_steps": 100,
  "expl_noise": 0.1,
  "batch_size": 256,
  "batches_before_train": 2,
  "evaluate_frequency": 5e3,
  "episode_max_steps":2500,
  "memory_capacity":10000,
  "num_episodes":5
})

ezai - I - Making empty DictObj because not given a dict or a json filename
2020-10-12 23:41:30 INFO [dict.py:41] Making empty DictObj because not given a dict or a json filename
ezai - I - Making empty DictObj because not given a dict or a json filename
2020-10-12 23:41:30 INFO [dict.py:41] Making empty DictObj because not given a dict or a json filename


In [8]:
trainer = airsim.Trainer(rl_conf,env,enable_logging=True)

2020-10-12 23:42:02 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0


(10,)


In [9]:
trainer.train()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]2020-10-12 23:42:10 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
2020-10-12 23:43:42 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
 20%|████████████████▌                                                                  | 1/5 [03:03<12:12, 183.06s/it]2020-10-12 23:45:13 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
2020-10-12 23:46:46 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
 40%|█████████████████████████████████▏                                                 | 2/5 [06:04<09:07, 182.63s/it]2020-10-12 23:48:14 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
2020-10-12 23:49:43 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0
 60%|█████████████████████████████████████████████████▊            

TypeError: cannot unpack non-iterable NoneType object

In [10]:
trainer.agent.save_actor('airsim.onnx')

In [11]:
env.close()

2020-10-13 09:45:02 INFO [environment.py:417] Environment shut down with return code 0 (CTRL_C_EVENT).
